In [ ]:
pip install plotly

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

# Data Cleaning

In [ ]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_store = pd.read_csv('data/store.csv')

In [ ]:
df_train.head()

In [ ]:
# Merge df_store and df_train
df = df_train.merge(df_store, how='left', left_on=df_train.Store, right_on=df_store.Store)
df.drop(['key_0', 'Store_y'], axis=1, inplace=True)
df = df.rename(columns={'Store_x':'Store'})
df.shape

In [ ]:
df.head()

In [ ]:
cols = df.columns.tolist()
cols = [col.lower() for col in cols]
df.columns = cols

In [ ]:
df.info()

In [ ]:
df = df.assign(
    timestamp = lambda x: pd.to_datetime(x['date']),
    year = lambda x: x['timestamp'].dt.year,
    month = lambda x: x['timestamp'].dt.month,
    day = lambda x: x['timestamp'].dt.day,
    dayofyear = lambda x: x['timestamp'].dt.dayofyear
)

In [ ]:
df.head()

In [ ]:
df['stateholiday'].unique()

In [ ]:
df.stateholiday.value_counts()

In [ ]:
df['stateholiday'].replace({0:'0'}, inplace=True)

In [ ]:
df.stateholiday.value_counts()

In [ ]:
round(df.describe().T,2)

### Handling NaN's

In [ ]:
df.isnull().sum()

In [ ]:
df_store[pd.isnull(df_store.CompetitionDistance)]

In [ ]:
# fill NaN with a median value
df['competitiondistance'].fillna(df['competitiondistance'].median(), inplace = True)
df['competitiondistance'].isnull().sum()

In [ ]:
tmp = df_store[pd.isnull(df_store.CompetitionOpenSinceYear  )]
tmp[tmp.CompetitionDistance != 0].shape

In [ ]:
tmp = df[pd.isnull(df.promo2sinceweek)]
tmp[tmp.promo2 != 0].shape

In [ ]:
# replace NA's by 0
df.fillna(0, inplace = True)

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df_test.isnull().sum()

In [ ]:
df_test[df_test["Open"].isnull()]

In [ ]:
df_test.fillna(1, inplace=True)

In [ ]:
df_test.isnull().sum()

### Handling Datatyp

In [ ]:
df_test["Open"]= df_test["Open"].astype(int)

In [ ]:
df_test.info()

### Handling Categorical Data

In [ ]:
le = LabelEncoder()

In [ ]:
ass_typ = pd.get_dummies(df["assortment"], drop_first=True)
df = pd.concat([df,ass_typ], axis=1)


In [ ]:
df["storetype_cat"] = le.fit_transform(df["storetype"])

In [ ]:
df["stateholiday_cat"] = le.fit_transform(df["stateholiday"])

In [ ]:
df =df.rename(columns={"b" : "ass_extra"})

In [ ]:
df =df.rename(columns={"c" : "ass_extended"})

In [ ]:
df["stateholiday_cat"].unique()

In [ ]:
df.info()

In [ ]:
df.stateholiday.value_counts()

In [ ]:
df.stateholiday_cat.value_counts()

# Eda

## Question: Are the Promos effective?

In [ ]:
df.groupby('storetype')['sales'].describe()

In [ ]:
df.groupby('storetype')['customers', 'sales'].sum()

In [ ]:
# sales trends
sns.factorplot(data = df, x = 'month', y = "sales", 
               col = 'storetype',
               palette = 'plasma',
               hue = 'storetype',
               row = 'promo', 
               ) 

## Finding

Storetype B has the highest sales numbers, with the largest variance. All storetypes show increased sales numbers towards christmas. 

Stores which have run a promo, show higher sales. But storetypes a,c and d show a dip towards easter, if they have run a promo, which is not the case for stores without a promo.

In [ ]:
# customers trends
sns.factorplot(data = df, x = 'month', y = "customers", 
               col = 'storetype',
               palette = 'plasma',
               hue = 'storetype',
               row = 'promo',
               ) 

## Finding

Storetype B has the highest number of customers, with the largest variance. All storetypes show an increase of customers towards christmas. This trend is higher, if they have run a promo.

Same effect of a dip for storetypes a,c and d in customers towards easter can be also be seen here.

In [ ]:
# sale per customer trends
df['salepercustomer'] = df['sales']/df['customers']
sns.factorplot(data = df, x = 'month', y = "salepercustomer", 
               col = 'storetype',
               palette = 'plasma',
               hue = 'storetype',
               row = 'promo', 
               ) 

## Finding

Sales per customer:
storetype b seems to be where customers only buy small items in low numbers (possible trainstation location?)
storetype d customers buy the largest quantity
a und c are very similar

In [ ]:
# weekday trends
sns.factorplot(data = df, x = 'dayofweek', y = "customers", 
               col = 'storetype',
               palette = 'plasma',
               hue = 'storetype',
               row = 'promo',
               ) 

## Finding

Promos are run only during the work-week, no promo on saturday/sunday.

Storetype b also open on sundays -> trainstation, fo sho
storetyp a lower number of customers on saturday, c and d increased

In [ ]:
# weekday trends
sns.factorplot(data = df, x = 'dayofweek', y = "sales", 
               col = 'storetype',
               palette = 'plasma',
               hue = 'storetype',
               row = 'promo',
               ) 

Similar trends regarding sales numbers and customers.
Highest number of sales and customers on mondays, if a promo was run

## Conclusion : Question: Are the Promos effective?"
- Promos are run only during the work-week, no promo on saturday/sunday\n",
- Storetype B has the highest number of customers, with the largest variance\n",
- Storetype B has the highest sales numbers, with the largest variance\n",
- All storetypes show increased sales numbers towards christmas\n",
- Stores which have run a promo, show higher sales. But storetypes a, c and d show a dip towards easter, if they have run a promo, which is not the case for stores without a promo."

In [ ]:
## beachte: date, (assortment, storetype und stateholiday,assortment) müssen noch raus. die in der klammer sind schon durch kategorische Variablen ersetzt

In [ ]:
sns.factorplot(data = df, x = 'month', y = "sales", 
               col = 'dayofweek', 
               palette = 'plasma',
               hue = 'storetype',
               row = 'storetype',
               ) 

This shows the sales per storetyp across all months for each weekday.

In [ ]:
# stores which are opened on Sundays
df[(df.open == 1) & (df.dayofweek == 7)]['store'].unique()

These are the stores, which are open on sundays.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(18,8))
sns.lineplot(x = "date", y = df.groupby(df.index).mean()["competitiondistance"] , data=df, ax=axes[0])
sns.lineplot(x = "date", y = df.groupby(df.index).mean()["competitionopensincemonth"]  , data=df, ax=axes[1])

axes[0].set_title("Date vs Competiton Distance (on average)")
axes[1].set_title("Date vs Competiton Open Since Month (on average)")
plt.show()

In [ ]:
temp_df = df.copy()
temp_df.index = temp_df.date
temp_df = temp_df.groupby(temp_df.index).mean()

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18,8))
sns.lineplot(x = "date", y = "competitiondistance" , data=temp_df, ax=axes[0])
sns.lineplot(x = "date", y = "competitionopensincemonth"  , data=df, ax=axes[1])

axes[0].set_title("Date vs Competiton Distance (on average)")
axes[1].set_title("Date vs Competiton Open Since Month (on average)")
plt.show()



In [ ]:
df_store.describe().T

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(18,8))
sns.lineplot(x = "Store", y = "CompetitionDistance" , data=df_store, ax=axes[0])
sns.lineplot(x = "Store", y = "CompetitionOpenSinceMonth"  , data=df_store, ax=axes[1])

In [ ]:
sns.set_style("dark")
sns.histplot(data=df_store, x="CompetitionDistance")


In [ ]:
sns.countplot(x="StoreType",hue="Assortment",order=["a","b","c","d"], 
                                     data=df_store,palette="rocket_r").set_title("Number of Different Assortments per Store Type")

In [ ]:
df.columns

In [ ]:
df['assperstoretype'] = df['assortment'] + df['storetype']

In [ ]:
temp = df.groupby('assperstoretype')['sales'].agg(['mean'])
temp

In [ ]:
#fig2 = px.scatter(df, x='store', y='sales', color='assperstoretype')
#fig2.show()

In [ ]:
#fig3 = px.scatter(df, x='store', y='sales', color='extraperstoretype')
#fig3.show()

In [ ]:
df['extendedperstoretype'] = (df['ass_extended']==1).astype(str) + '_' + df['storetype']

In [ ]:
#fig4 = px.scatter(df, x='store', y='sales', color='extendedperstoretype')
#fig4.show()

In [ ]:
#fig2.write_html("images/sales_store.html")
#fig3.write_html("images/sales_extraperstoretype.html")
#fig4.write_html("images/sales_extendedperstoretype.html")